In [ ]:
using HTTP
using JSON
using LargeScaleAnalysis
using PyCall
using PyPlot
using ThesisTools

### Helpers

In [ ]:
DateFormatter = pyimport("matplotlib.dates").DateFormatter
HourLocator = pyimport("matplotlib.dates").HourLocator;

In [ ]:
using Dates
function PyPlot.plot(index, data, state; ax = gca())
    cmap = Dict(i => plt.cm.tab20(i) for i in sort(unique(state)))
    ax.plot(unix2datetime.(index), coalesce.(data, NaN))
    segs = map(s -> maprange(s, index), segments(state, closed = true))
    for segment in segs
        span = unix2datetime.((segment.range.start, segment.range.stop))
        ax.axvspan(span..., alpha = 0.3, color = cmap[segment.state])
    end
end

In [ ]:
function fetch_json(url)
    r = HTTP.get(url)
    JSON.parse(String(r.body))
end

### Main

In [ ]:
res = fetch_json("https://trends.atlas.ripe.net/api/v1/trends/1437285/6222/?start=2018-05-02&stop=2018-05-10");

In [ ]:
index = map(x -> x[4], res["results"])
data = map(x -> x[5] > 0 ? x[5] : missing, res["results"])
state = map(x -> x[7], res["results"]);

In [ ]:
@pywith plt.style.context("classicthesis") begin
    fig, ax = subplots(figsize = (4.6, 0.35*4.6))
    plot(index, data, state, ax = ax)
    ax.set(yticks = [100,150,200], ylabel = "RTT (ms)", title = "at-vie-as1120 \$\\leftrightarrow\$ us-bos-as26167")
    ax.xaxis.set_major_formatter(DateFormatter("%d/%m\n%Hh%M"))
    fig.savefig("../figures/atlas-api-example.pdf")
end